In [99]:
import pandas as pd
from collections import defaultdict
from pprint import pprint

NUM_FORMULAS = 3
NUM_BRACKETS = 100
YEAR = 2018

def dfComp(oldFrame, newFrame):
    dfBool = (oldFrame == newFrame).stack()  
    sim = pd.concat([oldFrame.stack()[dfBool], newFrame.stack()[dfBool]], axis=1)
    sim.columns=["Predicted", "Actual"]
    return sim

def score_bracket(bracket_file,results_file):
    bracket_df = pd.read_csv(bracket_file,dtype='object')
    results_df = pd.read_csv(results_file,dtype='object')
    bracket_df = bracket_df.drop(columns='Rd1')
    results_df = results_df.drop(columns='Rd1')
    correct_df = dfComp(bracket_df, results_df)
    index = iter(correct_df.index.values)
    score = 0
    rdscores = {'Rd2':0,'S16':0,'E8':0,'FF':0,'C':0,'W':0}
    for ignore,rd in index:
        if rd == 'Rd2':
            rdscores['Rd2'] += 10
        elif rd == 'S16':
            rdscores['S16'] += 20
        elif rd == 'E8':
            rdscores['E8'] += 40
        elif rd == 'FF':
            rdscores['FF'] += 80
        elif rd == 'C':
            rdscores['C'] += 160
        elif rd == 'W':
            rdscores['W'] += 320
    return rdscores

# Score brackets
results_file = r'config\results.csv'
detailed_scores_file = r'scores\detailed_scores.csv'
top_brackets_file = r'scores\top_brackets.csv'
bracket_stats_file = r'scores\bracket_stats.csv'

all_scores = dict()
formula_scores = defaultdict(list)
for formula in range(1,NUM_FORMULAS+1):
    for bracket_id in range(1,NUM_BRACKETS+1):
        bracket_file =  r'brackets\formula' + str(formula) + '_' + str(YEAR) + '_' + str(bracket_id) + '.csv'
        scores = score_bracket(bracket_file, results_file)
        all_scores[bracket_file] = scores
        formula_scores[formula].append(sum(list(scores.values())))

# Prepare detailed_scores.csv   
detailed_scores_df = pd.DataFrame(all_scores)
detailed_scores_df = detailed_scores_df.T
detailed_scores_df['Total'] = detailed_scores_df.sum(axis=1)
detailed_scores_df = detailed_scores_df[['Total','Rd2','S16','E8','FF','C','W']]

write_csv1 = detailed_scores_df.to_csv (detailed_scores_file, header=True)

if write_csv1:
    print('Error with detailed_scores.csv')
else:
    print("detailed_scores.csv success")
    
# Prepare top_brackets.csv
top = 10
sorted_scores = sorted(detailed_scores_df['Total'].items(), reverse=True, key=lambda tup: tup[1])[:top]
top_brackets_df = pd.DataFrame(sorted_scores)
top_brackets_df = top_brackets_df.rename(columns={0:'Bracket File',1:'Score'})

write_csv2 = top_brackets_df.to_csv (top_brackets_file, index=None, header=True)

if write_csv2:
    print('Error with top_brackets.csv')
else:
    print("top_brackets.csv success")

# Prepare bracket_stats.csv
from statistics import mean, median, pstdev
formula_stats = dict()
for formula,scores in formula_scores.items():
    mn = mean(scores)
    md = median(scores)
    maxi = max(scores)
    mini = min(scores)
    std = pstdev(scores)
    formula_stats[formula] = [mn, md, maxi, mini, std]
    
bracket_stats_df = pd.DataFrame(formula_stats)
bracket_stats_df = bracket_stats_df.T
bracket_stats_df.index.name = 'Formula'
bracket_stats_df = bracket_stats_df.rename(columns={0:'Mean',1:'Median',2:'Maxi',3:'Mini',4:'Stdev'})

write_csv3 = bracket_stats_df.to_csv (bracket_stats_file, header=True)

if write_csv3:
    print('Error with bracket_stats.csv')
else:
    print("bracket_stats.csv success")

detailed_scores.csv success
top_brackets.csv success
bracket_stats.csv success
